In [1]:
import numpy as np
import os
import h5py
import matplotlib.pyplot as plt
import math
import sys
import time
from tqdm import tnrange, tqdm_notebook
import matplotlib.pyplot as plt
import operator
from matplotlib.backends.backend_pdf import PdfPages
import scipy.io
from matplotlib.gridspec import GridSpec
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize

# Neural Matrix

In [3]:
def make_reduced_neural_matrix(spike_train_file, image_times_file, save_dir):
    train_times = np.asarray(h5py.File(image_times_file, 'r')["image_times"])[0,:]
    train_images = train_times.shape[0]
    train_times = np.append(train_times, 9999)
    spike_train = np.load(spike_train_file)
    
    spike_times = spike_train[:,0] / 20000
    units = np.max(spike_train[:,1]).astype(np.int) + 1
    
    interval_add = np.arange(0.01, 0.5, 0.01)
    
    train_matrix = np.zeros((train_images, units, 50))
    bin_edges = np.copy(train_times)
    
    interval_add = np.arange(0.01, 0.5, 0.01)
    
    for i in tnrange(int(train_images)):
        for j in range(interval_add.shape[0]):
            bin_edge = train_times[i] + interval_add[j]
            bin_edges = np.append(bin_edges, bin_edge)

    bin_edges = np.sort(bin_edges)
    print(bin_edges.shape)
    
    hist_spikes, hist_edges = np.histogram(spike_times, bin_edges)
    
    count = spike_times.shape[0] - np.sum(hist_spikes)
    
    for i in tnrange(hist_spikes.shape[0]):
        bin_count = hist_spikes[i]
        for j in range(bin_count):  
            image = i//50
            unit = int(spike_train[count,1])
            index = i - 50*image
            train_matrix[image, unit, index] += 1
            count += 1
        
    print(count)
    print(spike_train.shape)
    
    #train_matrix = np.concatenate((train_matrix, np.ones((train_images, units, 1))), axis=2)
    
    test_matrix = train_matrix[9900:,:,:]
    valid_matrix = train_matrix[9800:9900, :,:]
    train_matrix = train_matrix[:9800,:,:]
    
    print(train_matrix.shape)
    print(test_matrix.shape)
    print(valid_matrix.shape)
    print(np.sum(train_matrix) + np.sum(test_matrix) + np.sum(valid_matrix))
    
    np.save(os.path.join(save_dir, "yass_50_train_neural.npy"), train_matrix)
    np.save(os.path.join(save_dir, "yass_50_test_neural.npy"), test_matrix)
    np.save(os.path.join(save_dir, "yass_50_valid_neural.npy"), valid_matrix)




In [4]:
spike_train_file = "/ssd/joon/2017_11_29_ns/yass/yass_spiketrain.npy"
image_times_file = "/ssd/joon/2017_11_29_ns/trigger_times_201711290.mat"
save_dir = "/ssd/joon/2017_11_29_ns/yass/neural/"

make_reduced_neural_matrix(spike_train_file, image_times_file, save_dir)



MemoryError: Unable to allocate array with shape (269571386,) and data type int32

# Initialize V = (k x 50), W = (p x nk)

In [10]:
def initialize_V_W(init_weights_file, k_dim, save_dir):
    init_weights = np.load(init_weights_file) #### ((2n+1) x p)
    unit_no = init_weights.shape[0]//2
    
    init_W = np.random.randn(95*146, unit_no * k_dim)
    
    for i in tnrange(unit_no):
        init_W[:, k_dim*i: k_dim*i+2] = init_weights[2*i : 2*i+2, :].T
    
    print(np.sum(init_weights))
    print(np.sum(init_W))
    
    init_V = np.random.randn(k_dim, 50)
    
    init_V[0,:] = 0
    init_V[0,3:16] = 1
    init_V[1,:] = 0
    init_V[1,17:31] = 1
    
    np.save(os.path.join(save_dir, "yass_init_V.npy"), init_V)
    np.save(os.path.join(save_dir, "yass_init_W.npy"), init_W)

In [11]:
init_weights_file = "/ssd/joon/2017_11_29_ns/yass/weights/yass_smooth_4_weights.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/reduced_rank/"
k_dim = 10

initialize_V_W(init_weights_file, k_dim, save_dir)


926567.5358418195
928800.5915259448


# Iterative Training

In [3]:
def V_W_iter_train(I, S, V, k_dim, unit_no, image_no, iteration):
    start_time = time.time()        
    ####### TRAIN W_NEW #####
    R = np.matmul(V, S)
    R = R.reshape((k_dim * unit_no, image_no))
        
    #IRT = np.matmul(I, R.T)
    #RRT = np.matmul(R, R.T)
    #RRT_inv = np.linalg.inv(RRT + 0.00001*np.identity(RRT.shape[0]))   
    #W_new = np.matmul(IRT, RRT_inv)
    
    W_new = np.matmul(I, np.linalg.pinv(R))
        
    ####### TRAIN V_NEW ######
    W = W_new
        
    #WTW = np.matmul(W.T, W)
    #WTW_inv = np.linalg.inv(WTW + 0.00001 * np.identity(WTW.shape[0]))
    #WTI = np.matmul(W.T, I)
    #R = np.matmul(WTW_inv, WTI)
    
    R = np.matmul(np.linalg.pinv(W), I)
    R = R.reshape((k_dim, unit_no * image_no))
        
    #RST = np.matmul(R, S.T)
    #SST = np.matmul(S, S.T)
    #SST_inv = np.linalg.inv(SST + 0.00001 * np.identity(SST.shape[0]))    
    #V_new = np.matmul(RST, SST_inv)
    
    V_new = np.matmul(R, np.linalg.pinv(S))
    V_new = normalize(V_new, axis=1, norm='l1')
        
    end_time = time.time()
        
    print(end_time - start_time)
        
    np.save(os.path.join(save_dir, "yass_" +str(iteration)+"_W.npy"), W_new)
    np.save(os.path.join(save_dir, "yass_" +str(iteration)+"_V.npy"), V_new)

def V_W_train(images_file, neural_file, init_V_file, iter_no, save_dir):
    
    images = np.load(images_file).T # (95 * 146, images)
    neural = np.load(neural_file) # (images, units, 50)
    init_V = np.load(init_V_file) # (k , 50)
    
    image_no = images.shape[1]
    unit_no = neural.shape[1]
    k_dim = init_V.shape[0]
    
    I = images - np.mean(images, axis = 1).reshape((images.shape[0], 1))
    S = neural - np.mean(neural, axis = 0).reshape((1, neural.shape[1], neural.shape[2]))
    S = S.reshape((50, image_no * unit_no))
    
    #V_array = np.empty((iter_no, k_dim, 50))
    #W_array = np.empty((iter_no, 95*146, unit_no * k_dim))
    
    for i in tnrange(iter_no):
        if i == 0:
            V_new, W_new = V_W_iter_train(I, S, init_V, k_dim, unit_no, image_no, i)
        else:
            V_new, W_new = V_W_iter_train(I, S, V_new, k_dim, unit_no, image_no, i)
        #V_array[i] = V_new
        #W_array[i] = W_new
        
    #np.save(os.path.join(save_dir, "yass_V_iterations.npy"), V_array)
    #np.save(os.path.join(save_dir, "yass_W_iterations.npy"), W_array)   
        
    

In [ ]:
images_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_50_train_neural.npy"
init_V_file = "/ssd/joon/2017_11_29_ns/yass/reduced_rank/yass_init_V.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/reduced_rank/"
iter_no = 1

V_W_train(images_file, neural_file, init_V_file, iter_no, save_dir)

In [ ]:
images_file = "/ssd/joon/2017_11_29_ns/images/smooth_train_images.npy"
neural_file = "/ssd/joon/2017_11_29_ns/yass/neural/yass_50_train_neural.npy"
init_V_file = "/ssd/joon/2017_11_29_ns/yass/reduced_rank/yass_init_V.npy"
save_dir = "/ssd/joon/2017_11_29_ns/yass/reduced_rank/"
iter_no = 1

V_W_train(images_file, neural_file, init_V_file, iter_no, save_dir)